In [6]:
from Hubbard.equalizer import *
import numpy as np
from Hubbard.io import *

N = 20
L = 4
Ly = 2
d = 3
shape = 'triangular'
w = 'x'
eqt = 'uvt'
R0 = np.array([3, 3, 7.2])
G = HubbardEqualizer(N,
                     R0=R0,
                     lattice=np.array([L, Ly], dtype=int),
                     band=1,
                     dim=d,
                     avg=0.5,
                     ghost=True,
                     sparse=True,
                     shape=shape,
                     waist=w,
                     equalize=False,
                     lattice_symmetry=False,
                     verbosity=0)
ghost = True
mask = np.ones(G.Nsite, dtype=bool)
err = ValueError('Ghost sites not implemented for this lattice.')


def xy_boundaries(G, N):
    x_bdry = np.concatenate((np.arange(N), np.arange(-N, 0)))
    y_bdry = np.concatenate(
        (np.arange(0, G.Nsite, N), np.arange(N-1, G.Nsite, N)))
    return x_bdry, y_bdry


if ghost:
    if G.lattice_dim == 1:
        mask[[0, -1]] = False
    elif G.lattice_dim == 2:
        if G.lattice_shape == 'square' \
                or G.lattice_shape == 'triangular' and not G.ls:
            Nx, Ny = G.size
            if G.lattice_shape == 'square':
                x_bdry, y_bdry = xy_boundaries(G, Ny)
            elif G.lattice_shape == 'triangular':
                y_bdry, x_bdry = xy_boundaries(G, Nx)
            bdry = [x_bdry, y_bdry]
            mask_axis = np.nonzero(G.size > 2)[0]
            if mask_axis.size != 0:
                masked_idx = np.concatenate([bdry[i] for i in mask_axis])
                mask[masked_idx] = False
        else:
            raise err
    else:
        raise err
    mask_idx = np.where(~mask)[0]
    G.masked_links = squeeze_idx(G.links, mask_idx)
else:
    G.masked_links = G.links
G.mask = mask


lattice: lattice shape is triangular; lattice constants set to: (1520, 1690)
Triangular lattice size adjust to: [4 2]
Equalize: scale_factor is not a number. Set to None.


In [4]:
from Hubbard.equalizer import *
import numpy as np
from Hubbard.io import *

N = 20
L = 4
Ly = 1
d = 3
shape = 'square'
w = 'x'
eqt = 'uvt'
R0 = np.array([3, 3, 7.2])
G = HubbardEqualizer(N,
                     R0=R0,
                     lattice=np.array([L, Ly], dtype=int),
                     band=1,
                     dim=d,
                     avg=0.5,
                     ghost=True,
                     sparse=True,
                     shape=shape,
                     waist=w,
                     equalize=False,
                     lattice_symmetry=True,
                     verbosity=0)
ghost = True
target = 'UvT'
u, t, v, fix_u, fix_t, fix_v = str_to_flags(target)
# Force corresponding factor to be 0 if flags u,t,v are false
weight: np.ndarray = np.array([u, t, v])

# Equalize trap depth first, to make sure traps won't go too uneven
# to have non-local WF. But this makes U to be more uneven.
# G.equalize_trap_depth()
# print(f"Equalize: trap depths equalzlied to {G.Voff}.")

A, U, V = G.singleband_Hubbard(u=u, offset=True)

# Set target to be already limited in the bulk
G.ghost_sites(ghost)
if G.masked_Nsite == 1:
    raise ValueError(
        "Equalize: only one site in the system, equalization is not valid.")
maskedA = A[G.mask, :][:, G.mask]
maskedU = U[G.mask] if u else None
links = G.xy_links(G.masked_links)
target = G._set_targets(None, fix_u, fix_t, links, maskedA, maskedU)
# Voff_bak = G.Voff
# ls_bak = G.trap_centers
# w_bak = G.waists

v0, bounds = G.init_guess()
init_simplx = None

G.eqinfo.create_log(v0, target)


lattice: lattice shape is square; lattice constants set to: (1520, 1690)
Equalize: scale_factor is not a number. Set to None.
Wannier WARNING: band gap is smaller than band width.
Equalize: scale factor = 0.2477400288970067
Equalize: target tunneling = 0.2477400288970067, None
Equalize: target interaction = 1.201462378588942
Equalize: target onsite potential = 0


In [5]:
v0

array([ 1.  ,  1.  ,  1.  ,  1.  , -2.28, -0.76])